# README

In [1]:
"""
Priority Grid Cells
Author: Liam Megraw, RIT Envirionmental Science Technician
Date last edited: 12/12/2022
ESRI ArcGIS Pro Version 2.7.0
Language: Python 3 (default environment)

Description:
This code generates socially and ecologically high-priority grid cells 
that contain invasive plant predictions from the RIT-developed computer 
vision model. Ecological factors come from the New York Natural Heritage
Program's spatial prioritization model comprehensive score. Social factors
come from the NY DEC Potential Environmental Justice Areas. Cells with 
all target plants already confirmed in the iMapInvasives database are 
removed. 

Inputs required, stored in one geodatabase:
> reporting_analysis_grid_empty (Fishnet grid for area of interest)
> comprehensive_score (NYNHP spatial prioritization raster)
> PEJA (NY DEC Potential Environmental Justice Area polygons)
> PRISM_Boundaries (NYNHP Regional management jurisdictions)
> pred_finalDeployment_all (Computer vision model results pre-thresholding)
> PRESENCE_POINT (iMap confirmed point data for species of interest)
> PRESENCE_LINE (iMap confirmed line data for species of interest)
> PRESENCE_POLYGON (iMap confirmed polygon data for species of interest)

Output:
The final output is a polygon layer with 1 km grid cells that are 
high verification priority, determined on a per-PRISM basis.

How to Use:
Once complete,
these grid cells should be hosted on the public AGOL dashboard to guide 
verification effort. If desired, manually select and remove any priority 
grid cells with known species present from past n years that have the
same species present as are predicted by the model. For the original output,
we removed any within the past 5 growing seasons.

Removing extra unnecessary fields is also desirable to save storage space 
and have a cleaner file (see current file in AGOL to identify necessary 
fields).
"""

"\nPriority Grid Cells\nAuthor: Liam Megraw, RIT Envirionmental Science Technician\nDate last edited: 11/17/2022\nESRI ArcGIS Pro Version 2.5.2\n\nDescription:\nThis code generates socially and ecologically high-priority grid cells \nthat contain invasive plant predictions from the RIT-developed computer \nvision model. Ecological factors come from the New York Natural Heritage\nProgram's spatial prioritization model comprehensive score. \n\nOutput:\nThe final output is a polygon layer with 1 km grid cells that are \nhigh verification priority, determined on a per-PRISM basis.\n\nHow to Use:\nOnce complete,\nthese grid cells should be hosted on the public AGOL dashboard to guide \nverification effort. If desired, manually select and remove any priority \ngrid cells with known species present from past n years that have the\nsame species present as are predicted by the model. For the original output,\nwe removed any within the past 5 growing seasons.\n\nRemoving extra unnecessary fields

In [2]:
"""
Pseudocode Overview

Define gdb location
Define inputs: grid cells, model predictions, comp score raster, PEJAs, PRISM bounds
    Copy ID to a new field
Create w x h minimum bounding rectangle per PRISM
    For each PRISM:
        Select all grid cells that intersect with a PRISM
        Create rectangle
        Select all grid cells within that rectangle
        Create temporary feature class
        Add feature class name to list
Calculate mean comprehensive score of grid cells 
    Loop for each grid cell feature in list:
        Comprehensive score zonal stats (mean), convert to int
Convert grid cells to vector & select high priority areas
    Loop for each grid cell feature in list:
        Perform identity function
        Calculate 90th percentile for PRISM
        Select cells above 90th percentile or in PEJAs
        Select cells that intersect model predictions
        Create new layer with only those cells
Merge all grid sets together
Only select cells with model data, then export
For each target plant:
    Spatial join model points to grid cells
    For each iMap geometry type:
        Spatial join iMap records to grid cells
    Sum join count across geometries
"""

'\nPseudocode Overview\n\nDefine gdb location\nDefine inputs: grid cells, model predictions, comp score raster, PEJAs, PRISM bounds\n    Copy ID to a new field\nCreate w x h minimum bounding rectangle per PRISM\n    Create empty list\n    For each PRISM:\n        Select all grid cells that intersect with a PRISM\n        Create rectangle\n        Select all grid cells within that rectangle\n        Create temporary feature class\n        Add feature class name to list\nCalculate mean comprehensive score of grid cells \n    Loop for each grid cell feature in list:\n        Comprehensive score zonal stats (mean), convert to int\nConvert grid cells to vector & select high priority areas\n    Loop for each grid cell feature in list:\n        Perform identity function\n        Calculate 90th percentile for PRISM\n        Select cells above 90th percentile or in PEJAs\n        Select cells that intersect model predictions\n        Create new layer with only those cells\nMerge all grid sets t

# Edits are necessary here

In [3]:
# Set environment -----
import arcpy
import os

arcpy.env.workspace = "C:\\Users\\ltmsbi\\Documents\\ArcGIS\\Projects\\Final_Deployment\\Final_Deployment.gdb"
arcpy.env.overwriteOutput = True

In [4]:
# Define Inputs, Thresholds & Other Parameters
# Everything should be in one projected coordinate system, preferably NAD 83 UTM 18N in line with what NYS uses
# Files outside of the gdb can be assigned using either the absolute path or relative path 
# If using relative paths, type ..\\ for each level up from the gdb directory
grid = "reporting_analysis_grid_empty" 
PRISM_Bounds = "PRISM_Boundaries"
comp_score = "Comprehensive_Score"
PEJAs = "PEJA"

# Put in the shortnames separated by commas from the all_PRISM_list (below) of the PRISMs you want to process results for
# Order must match between lists for naming functionality to work
# These are how names are defined by default in the PRISM boundary feature
input_PRISM_list = ["APIPP","Capital Region","CRISP",
            "Finger Lakes","Lower Hudson",
            "SLELO","Western NY"]

# These are the suffixes for various output items
PRISM_output_names = ["APIPP","Capital_Region","CRISP",
            "Finger_Lakes","Lower_Hudson",
            "SLELO","Western_NY"]
# Convert PRISM list into a tuple so PRISMs can be selected by the iteration number of the loop when calculating 90th percentile
PRISM_Tuple = tuple(PRISM_output_names)
PRISM_shortname_tuple = tuple(input_PRISM_list)

# Everything below *may* be left default

# Filepath for folder where you would like the merged priority grid shapefile
priority_grid_output_folder = "..\\" # By default, it'll export to the same folder as your geodatabase
merged_grid_name = "Priority_Grid_NYS" # Name for the final shapefile

# Define desired percentile of mean comprehensive score to be considered "high priority"
percentile = 90 # 90th percentile by default

# List of all PRISM shortnames as formatted in the PRISM Boundaries feature (not used for anything)
all_PRISM_list = ["APIPP","Capital Region","CRISP",
            "Finger Lakes","LIISMA","Lower Hudson",
            "SLELO","Western NY"]


# Edits should not be made below here

## Create minimum bounding rectangle grids

In [8]:
# Create minimum bounding rectangles (MBR) for each PRISM jurisdiction

# Copy ObjectID to new field if not done already
if 'Id' not in arcpy.ListFields(grid):
    print("Copying grid cell IDs")
    inTable = grid
    fieldName = 'Id'
    arcpy.management.AddField(inTable, fieldName, 'INTEGER')
    arcpy.management.CalculateField(inTable, fieldName, "!OBJECTID!", "PYTHON3") 
else:
    pass

# Set variables that will remain consistent for all loops
inputFeature = PRISM_Bounds
selectionType = "NEW_SELECTION"
fieldName = "SHORTNAME"
PRISMs = input_PRISM_list
# Create empty list to add filenames into
MBR_Grids = []

# For use in the deleting step of the loop
import os
cws = arcpy.env.workspace

# Create w x h grid cell sets within each PRISM
for i, PRISM in enumerate(PRISMs):
    print("Running for "+PRISM_Tuple[i]+" PRISM")
    # Select the PRISM
    print("***Selecting PRISM & grid cells...")
    whereClause = (fieldName + " = " + "'" + PRISM + "'")
    tmp_PRISM = arcpy.management.SelectLayerByAttribute(inputFeature, selectionType, whereClause)
    # Select grid cells within that PRISM
    tmp_PRISM_gc = arcpy.management.SelectLayerByLocation(grid, "INTERSECT", tmp_PRISM)
    # Create MBR
    print("***Creating feature of grid cells within MBR... ")
    outputFeature = ("tmpMBR")
    arcpy.management.MinimumBoundingGeometry(tmp_PRISM_gc, outputFeature, "RECTANGLE_BY_AREA", "ALL")
    # Select all grid cells within that MBR
    tmp_MBR_gc = arcpy.management.SelectLayerByLocation(grid, "INTERSECT", outputFeature)
    # Create feature of grid cells in that MBR
    outputFeature = ("Grid_" + PRISM_Tuple[i])
    arcpy.management.CopyFeatures(tmp_MBR_gc, outputFeature)
    # Add feature name to list
    MBR_Grids.append(outputFeature)
    
    # Clear selections for next run
    arcpy.management.SelectLayerByAttribute(inputFeature, "CLEAR_SELECTION")
    arcpy.management.SelectLayerByAttribute(grid, "CLEAR_SELECTION")
    
    # Delete temporary files and vars-----
    print("***Deleting temporary files...")
    del outputFeature
    # Get layer names as a list of strings
    inputs = ["tmpMBR"]
    # Remove each layer from contents view
    for input in inputs:
        arcpy.management.Delete(input)
        # This way is necessary to delete the feature itself and not just its contents
        input_path = os.path.join(cws, input)
        if arcpy.Exists(input_path):
            arcpy.Delete_management(input_path)

print("Grids created: ")
print(MBR_Grids) # Results in a list of grid features that should match the number of PRISMs specified in the PRISM_List variable

Copying grid cell IDs
Running for APIPP PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for Capital_Region PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for CRISP PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for Finger_Lakes PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for Lower_Hudson PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for SLELO PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Deleting temporary files...
Running for Western_NY PRISM
***Selecting PRISM & grid cells...
***Creating feature of grid cells within MBR... 
***Delet

## Create mean comprehensive score raster as integer

In [10]:
# Import spatial analyst tools for running zonal stats
from arcpy.sa import *

# Create mean comprehensive score raster 
for iteration, MBR_grid in enumerate(MBR_Grids):
    print("Running for " + PRISM_Tuple[iteration] + " PRISM")
    # Calculate mean comprehensive score per cell ID
    print("***Calculating mean comprehensive score per grid cell...")
    tmp_MCS = ZonalStatistics(MBR_grid, 'Id', comp_score, 'MEAN', 'DATA')
    # Convert to integer
    print("***Converting to integer...")
    tmp_MCS_int = Int(tmp_MCS)
    # Convert to vector 
    print("***Converting raster to polygon...")
    output_MCS_grid_name = ("MCS_int_"+PRISM_Tuple[iteration])
    arcpy.conversion.RasterToPolygon(tmp_MCS_int, output_MCS_grid_name, "NO_SIMPLIFY")
    
# Delete temporary files and vars-----
print("***Deleting temporary files...")
# Create list of layer names
inputs = [str(tmp_MCS), str(tmp_MCS_int)] + MBR_Grids
# Remove each layer from contents view
for input in inputs:
    arcpy.management.Delete(input)
del tmp_MCS, tmp_MCS_int, output_MCS_grid_name

print(PRISM_Tuple[iteration]+" PRISM complete")
print("All PRISMs processed!")

Running for APIPP PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for Capital_Region PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for CRISP PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for Finger_Lakes PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for Lower_Hudson PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for SLELO PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...
***Converting raster to polygon...
Running for Western_NY PRISM
***Calculating mean comprehensive score per grid cell...
***Converting to integer...

## Select priority grid cells for each PRISM

In [11]:
# Code to create list of vectorized mean comprehensive score grids
# Text to append onto prism names within the for loop
vector_grid_prefix = 'MCS_int_'
# Empty list for the loop to add values into
vector_grids = list()

# Loop to concatenate prefix with the PRISM, for use in the loop later on
for PRISM in PRISM_Tuple:
    vector_grids.append(vector_grid_prefix+PRISM)
print("List of grids to process: ")
print(vector_grids)

# Create empty dictionary to add percentiles into
# Using a dictionary instead of list so we can access percentile values later
pct_Dict = {"APIPP":[],"Capital_Region":[],"CRISP":[],
            "Finger_Lakes":[],"LIISMA":[],"Lower_Hudson":[],
            "SLELO":[],"Western_NY":[]
           }
# Create empty list to add names into for printing
priority_grids = list()


import numpy as np # Numpy used for percentile calculation
import os # Used for percentile calculation

# Used when selecting grid cells; PRISM_Bounds is defined in notebook cell 4
inputFeature = PRISM_Bounds
fieldName = "SHORTNAME"

# Create final vector grid & identify mean comprehensive score percentiles
print("Creating final grid for PRISMs: ")
print(PRISM_shortname_tuple)
for iteration, vector_grid in enumerate(vector_grids):
    print("Running for " + PRISM_Tuple[iteration] + " PRISM")
    # Select the PRISM
    print("***Selecting PRISM and grid cells...")
    whereClause = (fieldName + " = " + "'" + PRISM_shortname_tuple[iteration] + "'")
    tmp_PRISM = arcpy.management.SelectLayerByAttribute(inputFeature, "NEW_SELECTION", whereClause)
    # Select features that intersect that PRISM
    tmp_MCS_intersected = arcpy.SelectLayerByLocation_management(vector_grid, "HAVE_THEIR_CENTER_IN", tmp_PRISM)
    # Identify function 
    print("***Copying cell IDs...")
    tmpIdentityFeature = 'tmp_MCS_identity'
    arcpy.analysis.Identity(tmp_MCS_intersected, grid, tmpIdentityFeature, 'ALL')
    # Select only the individual grid cells mostly within a PRISM
    tmp_MCS_intermediate = arcpy.management.SelectLayerByLocation(tmpIdentityFeature, "HAVE_THEIR_CENTER_IN", tmp_PRISM)
    # Remove small feature artifacts
    arcpy.management.SelectLayerByAttribute(tmp_MCS_intermediate, "NEW_SELECTION", '"Shape_Area" >= 900000')
    arcpy.management.CopyFeatures(tmp_MCS_intermediate, 'tmp_MCS_final')
    tmp_MCS_final = 'tmp_MCS_final'
    # Get list of comprehensive score values
    print("***Calculating percentile...")
    N = [row.getValue('gridcode') for row in arcpy.SearchCursor(tmp_MCS_final)]
    # Run function defined above to calculate nth percentile for PRISM
    tmp_pct = np.percentile(N, percentile)
    """
    Add calculated percentile to list for viewing
    
    Selecting the entry from the PRISM tuple ensures the percentile value 
    is assigned correctly, regardless of the order the user inputs for the PRISMs
    """
    pct_Dict[PRISM_Tuple[iteration]].append(tmp_pct)
    # Keep only the grid cells with mean comprehensive score 
    # greater than or equal to the 90th percentile
    whereClause = ('"gridcode" >= '+str(tmp_pct))
    print("***Selecting cells where "+whereClause+"...")
    arcpy.management.SelectLayerByAttribute(tmp_MCS_final, 'NEW_SELECTION', whereClause)
    # Add grid cells intersecting PEJAs to selection
    print("***Selecting cells in PEJAs")
    tmp_MCS_hp = arcpy.management.SelectLayerByLocation(tmp_MCS_final, 'INTERSECT', PEJAs, "", "ADD_TO_SELECTION")
    # Define output name
    HPG = 'High_Priority_Grid_'+PRISM_Tuple[iteration]
    
    print("***Saving file...")
    # Save subsetted grid to gdb
    arcpy.management.CopyFeatures(tmp_MCS_hp, HPG)
    # Add names to list
    priority_grids.append(HPG)
    
    print("***Cleaning fields")
    arcpy.management.DeleteField(HPG, ['FID_reporting_analysis_grid_empty','FID_MCS_int_'+PRISM_Tuple[iteration],'Id'])
    arcpy.management.AlterField(HPG, 'Id_1','Id')
    
    # Delete temporary files and vars-----
    # Create list of layer names
    print("***Deleting temporary files...")
    inputs = [str(tmp_MCS_intersected), str(tmp_MCS_intermediate), str(tmp_MCS_final), str(tmp_MCS_hp), str(vector_grid)]
    # Remove each layer from contents view
    for input in inputs:
        arcpy.management.Delete(input)
        
    del tmp_MCS_intersected, tmp_MCS_intermediate, tmp_MCS_final, tmp_pct, tmp_MCS_hp, vector_grid

print("All done!")
print("Features created: ")
print(priority_grids)
print("Mean comprehensive score percentile per PRISM: ")
print(pct_Dict)

List of grids to process: 
['MCS_int_APIPP', 'MCS_int_Capital_Region', 'MCS_int_CRISP', 'MCS_int_Finger_Lakes', 'MCS_int_Lower_Hudson', 'MCS_int_SLELO', 'MCS_int_Western_NY']
Creating final grid for PRISMs: 
('APIPP', 'Capital Region', 'CRISP', 'Finger Lakes', 'Lower Hudson', 'SLELO', 'Western NY')
Running for APIPP PRISM
***Selecting PRISM and grid cells...
***Copying cell IDs...
***Calculating percentile...
***Selecting cells where "gridcode" >= 20.0...
***Selecting cells in PEJAs
***Saving file...
***Cleaning fields
***Deleting temporary files...
Running for Capital_Region PRISM
***Selecting PRISM and grid cells...
***Copying cell IDs...
***Calculating percentile...
***Selecting cells where "gridcode" >= 16.0...
***Selecting cells in PEJAs
***Saving file...
***Cleaning fields
***Deleting temporary files...
Running for CRISP PRISM
***Selecting PRISM and grid cells...
***Copying cell IDs...
***Calculating percentile...
***Selecting cells where "gridcode" >= 18.0...
***Selecting cells 

## Merge grid cells into one feature, save to GDB and selected output location

In [12]:
# Merge all grid cell features into one file 
# Output folder and name are defined in cell
priority_grid_merged = arcpy.management.Merge(priority_grids, merged_grid_name)
print("Geodatabase copy of "+merged_grid_name+" created")

Geodatabase copy of Priority_Grid_NYS created


In [22]:
# Delete all of the individual priority grids that were just merged
import os
cws = arcpy.env.workspace
for item in priority_grids:
    # Remove from contents view
    arcpy.management.Delete(item)
    input_path = os.path.join(cws, item)
    # Delete saved item
    if arcpy.Exists(input_path):
        arcpy.management.Delete(input_path)
del priority_grids

List of grids to process: 
['MCS_int_APIPP', 'MCS_int_Capital_Region', 'MCS_int_CRISP', 'MCS_int_Finger_Lakes', 'MCS_int_Lower_Hudson', 'MCS_int_SLELO', 'MCS_int_Western_NY']


## Remove cells with no model predictions

In [14]:
# Necessary input files
model_pred = "pred_finalDeployment_all"

# Only select cells with model data, then export
sel = arcpy.management.SelectLayerByLocation(priority_grid_merged, "INTERSECT", model_pred)
arcpy.management.CopyFeatures(sel, "tmp_PG1")

arcpy.management.Delete(sel)
del sel, model_pred

## Remove cells with all model target species already found

In [15]:
# Create list of imap features to iterate over
imap_data = list() #[imap_points, imap_lines, imap_polygons]
geometries = ["POINT","LINE","POLYGON"]
types = ["CONFIRMED"]
for geometry in geometries:
    for record_type in types:
        if record_type is "CONFIRMED":
            imap_data.append("PRESENCE_"+geometry)
        else:
            imap_data.append("PRESENCE_"+geometry+"_"+record_type)

print("iMap datasets:")
print(imap_data)

# Create dictionary of long names
# Names used for filtering in ArcGIS Online
species_fullnames = {
    "phrag": "'Phragmites, Unspecified'", # extra sinlge quotes are intentional since these are used in a field calculation
    "knot": "'Knotweed, Unspecified'",
    "wp": "'Wild Parsnip'",
    "toh": "'Tree-of-Heaven (Ailanthus)'",
    "pl": "'Purple Loosestrife'"
}

# Extract only the keys to a list
species_names = species_fullnames.keys()

# IDs that iMap assigns to the various species of interest
jurisdiction_ids = {
    "phrag": 1277,
    "wp": 1182,
    "pl": 1265,
    "toh": 1167,
    "knot": (1074, 1191, 1278, 1479) # Includes Japanese knotweed, giant knotweed, bohemian knotweed, and knotweed species unknown
}

# Per-species gap analysis with thresholded data
dataset_lists = [imap_data]
keepFields = list() # List to add fields into to keep in the final output
tmpFeatures = list()

# Define target feature for first run
targetFeature = "tmp_PG1"

for datasets in dataset_lists:
    for species in species_names:
        # For the individual feature in each dataset list
        # Create join count features
        for joinFeature in datasets:
            print(joinFeature)
            
            # Select just the species
            
            # If model positives ----------

            # Code to select 4 knotweed IDs
            if species is "knot":
                # Set initial SQL query
                whereClause = "jurisdiction_species_id = "+str(jurisdiction_ids["knot"][0])
                # Add more conditions to query
                for ID in jurisdiction_ids["knot"][1:]:
                    whereClause = whereClause + " Or jurisdiction_species_id = " + str(ID)
            # Otherwise, just select the single species ID
            else:
                whereClause = "jurisdiction_species_id = "+str(jurisdiction_ids[species])

            # Define unique output names
            if "POINT" in joinFeature:
                outFeature = "tmpRAG"+species+"_point"+"_cnfrm"
                fieldName = "imap_point_"+species+"_cnfrm"
            elif "LINE" in joinFeature:
                outFeature = "tmpRAG"+species+"_line"+"_cnfrm"
                fieldName = "imap_line_"+species+"_cnfrm"
            elif "POLYGON" in joinFeature:
                outFeature = "tmpRAG"+species+"_poly"+"_cnfrm"
                fieldName = "imap_poly_"+species+"_cnfrm"
            # ----------
            
            # Add field names to list of what fields to keep
            keepFields.append(fieldName)
            # Add feature to list to delete later
            tmpFeatures.append(outFeature)
            
            # Make species selection
            sel = arcpy.management.SelectLayerByAttribute(joinFeature, "NEW_SELECTION", whereClause)
                
            # Count the features within each grid cell
            arcpy.analysis.SpatialJoin(targetFeature, sel, outFeature)
            
            # Clear selection for next run
            arcpy.management.SelectLayerByAttribute(joinFeature, "CLEAR_SELECTION")
            
            # Rename the join field
            arcpy.management.AlterField(outFeature, "JOIN_COUNT", fieldName, fieldName)
            
            # Make the output feature the input for the next join
            targetFeature = outFeature

# Delete unnecessary fields
# keepFields is iteratively appended above
print("Deleting extra fields")
keepFields = keepFields + ['OBJECTID', 'Shape', 'Shape_Area', 'Shape_Length', 'Id']
allFields = [f.name for f in arcpy.ListFields(outFeature)]
# Get a list of all fields *except* the ones we want to keep
deleteFields = list(set(allFields) - set(keepFields))
arcpy.management.DeleteField(outFeature, deleteFields)

# Re-name the final output feature that has all desired fields
PG2 = "tmp_PG2"
arcpy.management.CopyFeatures(outFeature, PG2)

iMap datasets:
['PRESENCE_POINT', 'PRESENCE_LINE', 'PRESENCE_POLYGON']
PRESENCE_POINT
PRESENCE_LINE
PRESENCE_POLYGON
PRESENCE_POINT
PRESENCE_LINE
PRESENCE_POLYGON
PRESENCE_POINT
PRESENCE_LINE
PRESENCE_POLYGON
PRESENCE_POINT
PRESENCE_LINE
PRESENCE_POLYGON
PRESENCE_POINT
PRESENCE_LINE
PRESENCE_POLYGON
Deleting extra fields


<Result 'C:\\Users\\ltmsbi\\Documents\\ArcGIS\\Projects\\Final_Deployment\\Final_Deployment.gdb\\tmp_PG2'>

In [16]:
# Summary calculations ----------

# Create lists to use for calculating per-species totals

geometries = ["point","line","poly"]
types = ["cnfrm"]

sum_dict = dict() # filled in with 10 key-value pairs with loop below

for species in species_names:
    for record_type in types:
        tmpList = list()
        for geometry in geometries:
            sum_dict.update({"imap_"+species+"_"+record_type: []})
            
            tmpList.append("!imap_"+geometry+"_"+species+"_"+record_type+"!")
        sum_dict["imap_"+species+"_"+record_type].append(tmpList)
# -----

# For iMap data, calculate per-species join count sums
for datasets in dataset_lists:
    print("Calculating per-species iMap features joined")
    for species in species_names:
        print(species)
        for record_type in types:
            # Sum per-species features across all geometry types for each record type
            # Calculate the total number of iMap features joined
            print("***Adding count fields")
            rName = "iMap_"+record_type+"_"+species
            arcpy.management.AddField(PG2, rName, 'LONG')

            # Unpack list in dictionary entries
            sum_fields = [x for l in sum_dict['imap_'+species+"_"+record_type] for x in l]

            # Create expression for calculating sum of all geometry types
            express = sum_fields[0]+"+"+sum_fields[1]+"+"+sum_fields[2]

            print("***Summing")
            arcpy.management.CalculateField(PG2, rName, express)
    # ----------

Calculating per-species iMap features joined
phrag
***Adding count fields
***Summing
knot
***Adding count fields
***Summing
wp
***Adding count fields
***Summing
toh
***Adding count fields
***Summing
pl
***Adding count fields
***Summing


In [17]:
# Remove cells with all model target species identified
whereAllSpecies = ""
for i, species in enumerate(species_names):
    if i < len(species_names)-1:
        whereAllSpecies = whereAllSpecies+"iMap_cnfrm_"+species+" > 0 And "
    elif i is len(species_names)-1:
        whereAllSpecies = whereAllSpecies+"iMap_cnfrm_"+species+" > 0"
        
# Select cells to keep and export to new file
sel = arcpy.management.SelectLayerByAttribute(PG2, "NEW_SELECTION", whereAllSpecies, "INVERT")
finalGrid = "Priority_Grid_NYS_Final"
arcpy.management.CopyFeatures(sel, finalGrid)

print("Deleting extra fields")
keepFields = ['OBJECTID', 'Shape', 'Shape_Area', 'Shape_Length', 'Id']
allFields = [f.name for f in arcpy.ListFields(finalGrid)]
# Get a list of all fields *except* the ones we want to keep
deleteFields = list(set(allFields) - set(keepFields))
arcpy.management.DeleteField(finalGrid, deleteFields)

print("Done!")

Deleting extra fields
Done!


In [18]:
# Delete temp files
import os
deleteItems = ["Priority_Grid_NYS", "tmp_PG1", "tmp_PG2"] + tmpFeatures # tmpFeatures contains all of the iMap items
for item in deleteItems:
    # Remove from contents view
    arcpy.management.Delete(item)
    input_path = os.path.join(cws, item)
    # Delete saved item
    if arcpy.Exists(input_path):
        arcpy.management.Delete(input_path)
        
# Delete all variables
# initializing d with dir()
d = dir()

#You'll need to check for user-defined variables in the directory
for obj in d:
    #checking for built-in variables/functions
    if not obj.startswith('__'):
        #deleting the said obj, since a user-defined function
        del globals()[obj]